In [1]:
from lxml import etree

In [2]:
path = '/media/ilya/My Passport/HSE/VKR/ML_AOI_helper/rawDATA/all-vvts-xml-marchonly/Y1MTB44B5Z05F#MBDX86742001B_T#V510#D-SMT-AOI-41#1#1709274502000#PostRepair#1709276140000.xml'
root = etree.parse(path)
root, root.getroot()

(<lxml.etree._ElementTree at 0x755523dec1c0>,
 <Element {http://tempuri.org/BoardTestXMLExport.xsd}BoardTestXMLExport at 0x755523def000>)

In [3]:
root.xpath('//BoardTestXMLExport/text', namespaces=root.getroot().nsmap)

[]

In [4]:
root.getroot().tag

'{http://tempuri.org/BoardTestXMLExport.xsd}BoardTestXMLExport'

In [5]:
ns = root.getroot().nsmap
ns

{'ns1': 'http://tempuri.org/BoardTestXMLExport.xsd'}

In [143]:
sn = root.getroot().find("ns1:BoardXML", ns).attrib.get('serialNumber')
sn

'Y1MTB44B5Z05F'

In [52]:
root.getroot().find('BoardXML', namespaces=root.getroot().nsmap)

In [6]:
rootaa = etree.XML("""<ns1:root xmlns:ns1="http://tempuri.org/BoardTestXMLExport.xsd" ><ns1:a x='123'>aText<b/><c/><b/></ns1:a></ns1:root>""")
rootaa

<Element {http://tempuri.org/BoardTestXMLExport.xsd}root at 0x755520427ac0>

In [7]:
rootaa.find(".//ns1:a", ns).text

'aText'

In [8]:
tests = root.getroot().findall(".//ns1:TestXML", ns) #.attrib.get('name')
len(tests)

74

In [9]:
aa = tests[0].findall(".//ns1:IndictmentXML", ns)
aa

[<Element {http://tempuri.org/BoardTestXMLExport.xsd}IndictmentXML at 0x755509ba5600>,
 <Element {http://tempuri.org/BoardTestXMLExport.xsd}IndictmentXML at 0x755509ba6dc0>]

In [10]:
aaa = aa[0].find("ns1:RepairActionXML", ns)
aaa

<Element {http://tempuri.org/BoardTestXMLExport.xsd}RepairActionXML at 0x755509ba6c00>

In [24]:
class Defect:
    assemblyRevision = ''
    serialNumber = ''
    testerName = ''
    type = ''
    indictmentType = ''
    repairOperator = ''
    repairStatus = ''
    designator = ''
    partId = ''
    pinId = ''

    def to_dict(self):
        res = {}
        res['assemblyRevision'] = self.assemblyRevision
        res['serialNumber'] = self.serialNumber
        res['testerName'] = self.testerName
        res['type'] = self.type
        res['indictmentType'] = self.indictmentType
        res['repairOperator'] = self.repairOperator
        res['repairStatus'] = self.repairStatus
        res['designator'] = self.designator
        res['partId'] = self.partId
        res['pinId'] = self.pinId

        return res

In [25]:
def parse_file_for_defects(path):
    root = etree.parse(path)
    ns = root.getroot().nsmap
    res = [] #Defect()
    serialNumber = root.getroot().find("ns1:BoardXML", ns).attrib.get('serialNumber')
    testerName = root.getroot().find("ns1:StationXML", ns).attrib.get('testerName')
    assemblyRevision = root.getroot().find("ns1:BoardXML", ns).attrib.get('assemblyRevision')

    # TestXML = root.getroot().findall(".//ns1:TestXML", ns)
    TestXML = root.getroot().findall(".//ns1:TestXML", ns)
    for test in TestXML:
        name = test.attrib.get('name')
        IndictmentXML = test.findall(".//ns1:IndictmentXML", ns)
        PinXML = test.findall(".//ns1:PinXML", ns)
    
        for Indictment in IndictmentXML:
            tmp = Defect()
            tmp.assemblyRevision = assemblyRevision
            tmp.serialNumber = serialNumber
            tmp.testerName = testerName
            tmp.designator = name
            tmp.type = 'IndictmentXML'
            tmp.indictmentType = Indictment.attrib.get('indictmentType')
            tmp.repairOperator = Indictment.find("ns1:RepairActionXML", ns).attrib.get('repairOperator')
            tmp.repairStatus = Indictment.find("ns1:RepairActionXML", ns).attrib.get('repairStatus')
            tmp.partId = Indictment.find("ns1:ComponentXML", ns).attrib.get('partId')
            res.append(tmp)

        partId = IndictmentXML[0].find("ns1:ComponentXML", ns).attrib.get('partId')
        for pin in PinXML:
            repairStatus = pin.attrib.get('repairStatus')
            id = pin.attrib.get('id')
            PinIndictmentXML = pin.findall(".//ns1:PinIndictmentXML", ns)

            for pinIndictment in PinIndictmentXML:
                tmp = Defect()
                tmp.assemblyRevision = assemblyRevision
                tmp.serialNumber = serialNumber
                tmp.testerName = testerName
                tmp.designator = name
                tmp.type = 'PinXML'
                tmp.indictmentType = Indictment.attrib.get('indictmentType')
                tmp.repairOperator = Indictment.find("ns1:RepairActionXML", ns).attrib.get('repairOperator')
                tmp.repairStatus = repairStatus
                tmp.partId = partId
                tmp.pinId = id
                res.append(tmp)
        
    return res


In [26]:
file_1 = parse_file_for_defects(path)
len(file_1)

348

In [27]:
file_1[200].to_dict()

{'assemblyRevision': 'MBDX86742001B_T',
 'serialNumber': 'Y1MTB44B5Z05F',
 'testerName': 'D-SMT-AOI-41',
 'type': 'PinXML',
 'indictmentType': 'Coplanarity',
 'repairOperator': 'Svetlana_VVTS08',
 'repairStatus': 'False Call',
 'designator': '1:c22',
 'partId': 'GRM188C80G106KE47D',
 'pinId': '0'}

In [29]:
from os import listdir
mypath = '/media/ilya/My Passport/HSE/VKR/ML_AOI_helper/rawDATA/all-vvts-xml-marchonly/'
all_files = listdir(mypath)

In [30]:
all_files[4]

'Y2G9A44BYF00D#MBDX86691004A_T#V510#D-SMT-AOI-51#1#1709316486000#PostRepair#1709317954000.xml'

In [33]:
all_defects = []

for file in all_files:
    tmp = parse_file_for_defects(mypath + file)
    all_defects.extend(tmp)

In [34]:
len(all_defects)

354537

In [39]:
all_defects[200002].to_dict()

{'assemblyRevision': 'MBDX86742001B_T',
 'serialNumber': 'Y1MTB44BYF01E',
 'testerName': 'D-SMT-AOI-41',
 'type': 'PinXML',
 'indictmentType': 'Polarity',
 'repairOperator': 'Larisa_VVTS08',
 'repairStatus': 'False Call',
 'designator': '1:d91',
 'partId': 'QTLP601CRTR',
 'pinId': '0'}